# MFA Audit of Users with Streamlit in Snowflake Notebooks 📓

Ever wondered which of your users have MFA enabled and for those who have not, we can retrieve a list of those users and have it delivered straight to your email inbox. 

Conceptually, we'll perform the following tasks in this notebook:
- Generate an artificial user dataset
- Craft a query to display a DataFrame consisting of user ID, email and MFA status
- Create a conditional button that emails a system administrator a formatted table specifying which users who do not have MFA enabled

## Creating the User Data Set

In this notebook, we'll use an artificially generated [user dataset](https://sfquickstarts.s3.us-west-1.amazonaws.com/sfguide_building_mfa_audit_system_with_streamlit_in_snowflake_notebooks/demo_data.csv), from which we'll retrieve a subset of columns to display (e.g. `USER_ID`, `LOGIN_NAME`, `EMAIL` and `HAS_MFA`).

### Approach 1: Creation via SQL Query
For this first approach, we'll setup and create via SQL query.

The following query sets up the necessary administrative permissions, compute resources, database structures, and data staging areas to load MFA user data from an external S3 bucket.

In [ ]:
USE ROLE ACCOUNTADMIN; -- Sets current role to ACCOUNTADMIN
CREATE OR REPLACE WAREHOUSE MFA_DEMO_WH; -- By default, this creates an XS Standard Warehouse
CREATE OR REPLACE DATABASE MFA_DEMO_DB;
CREATE OR REPLACE SCHEMA MFA_DEMO_SCHEMA;
CREATE OR REPLACE STAGE MFA_DEMO_ASSETS; -- Store data files

-- create csv format
CREATE FILE FORMAT IF NOT EXISTS MFA_DEMO_DB.MFA_DEMO_SCHEMA.CSVFORMAT 
    SKIP_HEADER = 1 
    TYPE = 'CSV';

-- Create stage and load external demo data from S3
CREATE STAGE IF NOT EXISTS MFA_DEMO_DB.MFA_DEMO_SCHEMA.MFA_DEMO_DATA 
    FILE_FORMAT = MFA_DEMO_DB.MFA_DEMO_SCHEMA.CSVFORMAT 
    URL = 's3://sfquickstarts/sfguide_building_mfa_audit_system_with_streamlit_in_snowflake_notebooks/demo_data.csv';
    -- https://sfquickstarts.s3.us-west-1.amazonaws.com/sfguide_building_mfa_audit_system_with_streamlit_in_snowflake_notebooks/demo_data.csv

LS @MFA_DEMO_DATA; -- List contents of the stage we just created

Next, we'll copy the staged data from an S3 bucket into a newly created `MFA_DATA` table.

In [ ]:
-- Create a new data table called MFA_DEMO
CREATE OR REPLACE TABLE MFA_DEMO_DB.MFA_DEMO_SCHEMA.MFA_DATA (
    USER_ID NUMBER,
    NAME VARCHAR(100),
    CREATED_ON TIMESTAMP,
    DELETED_ON TIMESTAMP,
    LOGIN_NAME VARCHAR(100),
    DISPLAY_NAME VARCHAR(100),
    FIRST_NAME VARCHAR(50),
    LAST_NAME VARCHAR(50),
    EMAIL VARCHAR(255),
    MUST_CHANGE_PASSWORD BOOLEAN,
    HAS_PASSWORD BOOLEAN,
    COMMENT VARCHAR(255),
    DISABLED BOOLEAN,
    SNOWFLAKE_LOCK BOOLEAN,
    DEFAULT_WAREHOUSE VARCHAR(100),
    DEFAULT_NAMESPACE VARCHAR(100),
    DEFAULT_ROLE VARCHAR(100),
    EXT_AUTHN_DUO BOOLEAN,
    EXT_AUTHN_UID VARCHAR(100),
    HAS_MFA BOOLEAN,
    BYPASS_MFA_UNTIL TIMESTAMP,
    LAST_SUCCESS_LOGIN TIMESTAMP,
    EXPIRES_AT TIMESTAMP,
    LOCKED_UNTIL_TIME TIMESTAMP,
    HAS_RSA_PUBLIC_KEY BOOLEAN,
    PASSWORD_LAST_SET_TIME TIMESTAMP,
    OWNER VARCHAR(100),
    DEFAULT_SECONDARY_ROLE VARCHAR(100),
    TYPE VARCHAR(50)
);

-- Copy the data from your stage to this newly created table
COPY INTO MFA_DEMO_DB.MFA_DEMO_SCHEMA.MFA_DATA
    FROM @MFA_DEMO_DB.MFA_DEMO_SCHEMA.MFA_DEMO_DATA

### Approach 2: Creation via GUI
As for the second approach, we'll upload the [user dataset](https://github.com/Snowflake-Labs/snowflake-demo-notebooks/blob/main/MFA%20Audit%20of%20Users/demo_data.csv) to Snowflake by clicking on `+` --> `Table` --> `From File` (left sidebar menu) and create a table called `CHANINN_DEMO_DATA.PUBLIC.MFA_DATA`.

## Displaying the User Data Set

Next, we'll use the following SQL query to retrieve and display the user dataset. Particularly, we're displaying a subset of the data where `HAS_MFA` is `FALSE`, which translates to users who do not have MFA activated.

In [ ]:
SELECT USER_ID, LOGIN_NAME, EMAIL, HAS_MFA
FROM CHANINN_DEMO_DATA.PUBLIC.MFA_DATA
WHERE HAS_MFA = 'FALSE'

## Creating a Notification Integration

A notification integration is a Snowflake object that provides an interface between Snowflake and third-party messaging services (*e.g.* third-party cloud message queuing services, email services, webhooks, etc.). 

In a nutshell, this allows us to perform the necessary setup for sending an email notification that we'll do in the subsequent phase of this notebook.

In [ ]:
CREATE OR REPLACE NOTIFICATION INTEGRATION my_email_int
  TYPE=EMAIL
  ENABLED=TRUE
;

## Sending a Test Message

Here, we'll send a simple test notification using the `CALL SYSTEM$SEND_EMAIL()` stored procedure.

Note: Please replace `your-name@email-address.com` with your email address.

In [ ]:
CALL SYSTEM$SEND_EMAIL(
    'my_email_int',
    'your-name@email-address.com',
    'Email subject goes here',
    'Hello world! This is a test message!'
);

## Interactively Send MFA Status

In this simple example, we'll collate a table of users who has not activated their MFA then emailing this to a system administrator (*i.e.* you or an actual system administrator).

We'll make this interactive by placing a button (via `st.button()`) as a conditional trigger that runs downstream code upon a user clicking on them.

Finally, the SQL command, `SYSTEM$SEND_EMAIL` is run to send an email notification that is essentially a table of users who has not activated MFA.

Note: Please replace `your-name@email-address.com` with your email address.

In [ ]:
from snowflake.snowpark.context import get_active_session
import streamlit as st

session = get_active_session()

# DataFrame of users and their MFA status
st.header('MFA activation status')

mfa_selection = st.selectbox('Select an MFA status:', ('All', 'MFA Activated', 'MFA Not Activated'))
if mfa_selection == 'All':
    df = session.sql(
              """SELECT USER_ID, LOGIN_NAME, EMAIL, HAS_MFA 
                    FROM CHANINN_DEMO_DATA.PUBLIC.MFA_DATA"""
            ).to_pandas()
    paragraph = "<p>Here's the Multi-Factor Authentication status of all users. Please refer users to the <a href='https://docs.snowflake.com/en/user-guide/security-mfa'>Docs page on MFA</a> to activate MFA.</p>"
if mfa_selection == 'MFA Activated':
    df = session.sql(
              "SELECT USER_ID, LOGIN_NAME, EMAIL, HAS_MFA FROM CHANINN_DEMO_DATA.PUBLIC.MFA_DATA WHERE HAS_MFA = 'TRUE'"
            ).to_pandas()
    paragraph = "<p>Congratulations, these users have activated their Multi-Factor Authentication!</p>"
if mfa_selection == 'MFA Not Activated':
    df = session.sql(
              "SELECT USER_ID, LOGIN_NAME, EMAIL, HAS_MFA FROM CHANINN_DEMO_DATA.PUBLIC.MFA_DATA WHERE HAS_MFA = 'FALSE'"
            ).to_pandas()
    paragraph = "<p>It appears that the following users have not activated Multi-Factor Authentication. Please refer users to the <a href='https://docs.snowflake.com/en/user-guide/security-mfa'>Docs page on MFA</a> to activate MFA.</p>"
st.dataframe(df)

# Send Email
if st.button('Send Report'):
    email= 'your-name@email-address.com'
    email_subject = "Important: Activate Multi-Factor Authentication for User's Account"
    header = '<p>Dear System Administrator,</p>'
    body = header + '\n' + paragraph + '\n' + df.to_html(index=False, justify='left')

    session.call('SYSTEM$SEND_EMAIL',
                             'my_email_int',
                              email,
                              email_subject,
                              body,
                             'text/html')
    st.success('Report sent!', icon='✅')

## Resources
If you'd like to take a deeper dive into customizing the notebook, here are some useful resources to get you started.
- [Multi-factor authentication (MFA)](https://docs.snowflake.com/en/user-guide/security-mfa)
- [Sending email notifications](https://docs.snowflake.com/en/user-guide/notifications/email-notifications)
- [SYSTEM$SEND_EMAIL](https://docs.snowflake.com/en/sql-reference/stored-procedures/system_send_email)
- [Using SYSTEM$SEND_EMAIL to send email notifications](https://docs.snowflake.com/en/user-guide/notifications/email-stored-procedures)